In [ ]:
from google.colab import drive
import sys

drive.mount('/content/drive')

base_dir = '/content/drive/MyDrive/HYU/2025/AUE8088/project/AUE8088/'
!cd /content/drive/MyDrive/HYU/2025/AUE8088/project/AUE8088/

sys.path.append(base_dir)

Mounted at /content/drive


# Jupyter notebook for debugging

In [ ]:
%load_ext autoreload
%autoreload 2

# Copied from `train` function in train_simple.py:L78
import yaml

device = 'cpu'
hyp = base_dir + 'data/hyps/hyp.scratch-low.yaml'

with open(hyp, errors="ignore") as f:
    hyp = yaml.safe_load(f)  # load hyps dict

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
from models.yolo import Model
from utils.general import check_dataset

cfg = base_dir + 'models/yolov5n_nuscenes.yaml'
data = base_dir + 'data/nuscenes.yaml'
data_dict = check_dataset(data)

nc = int(data_dict["nc"])  # number of classes
model = Model(cfg, ch=3, nc=nc, anchors=hyp.get("anchors")).to(device)  # create

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.



Dataset not found ⚠️, missing paths ['/content/drive/MyDrive/HYU/2025/AUE8088/project/AUE8088/datasets/nuscenes/val.txt']
Unzipping /content/drive/MyDrive/HYU/2025/AUE8088/project/AUE8088/datasets/nuscenes/images/VOCtest_06-Nov-2007.zip...
Unzipping /content/drive/MyDrive/HYU/2025/AUE8088/project/AUE8088/datasets/nuscenes/images/VOCtrainval_06-Nov-2007.zip...
Unzipping /content/drive/MyDrive/HYU/2025/AUE8088/project/AUE8088/datasets/nuscenes/images/VOCtrainval_11-May-2012.zip...
test2007: 100%|██████████| 4952/4952 [01:09<00:00, 71.16it/s]
Dataset download success ✅ (1343.4s), saved to /content/drive/MyDrive/HYU/2025/AUE8088/project/datasets
100%|██████████| 755k/755k [00:00<00:00, 18.3MB/s]
Overriding model.yaml nc=4 with nc=15

                 from  n    params  module                                  arguments                     
  0                -1  1      1760  models.common.Conv                      [3, 16, 6, 2, 2]              
  1                -1  1      4672  models.co

In [ ]:
anchors = model.model[-1].anchors

# [TODO] Draw anchors

In [ ]:
from utils.dataloaders import create_dataloader
from utils.general import check_img_size, colorstr

imgsz = 416
batch_size = 1
single_cls = False
seed = 0

train_path = '/content/drive/MyDrive/HYU/2025/AUE8088/project/AUE8088/datasets/nuscenes/images/train2007' # data_dict["train"]
gs = max(int(model.stride.max()), 32)  # grid size (max stride)
imgsz = check_img_size(imgsz, gs, floor=gs * 2)  # verify imgsz is gs-multiple

train_loader, dataset = create_dataloader(
    train_path,
    imgsz,
    batch_size,
    gs,
    single_cls,
    hyp=hyp,
    augment=True,
    cache=None,
    rect=False,
    rank=-1,
    workers=8,
    image_weights=False,
    quad=False,
    prefix=colorstr("train: "),
    shuffle=True,
    seed=seed,
)

albumentations: 1 validation error for InitSchema
size
  Field required [type=missing, input_value={'scale': (0.8, 1.0), 'ra...: None, 'strict': False}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing
train: Scanning /content/drive/MyDrive/HYU/2025/AUE8088/project/AUE8088/datasets/nuscenes/labels/train2007... 2500 images, 1955 backgrounds, 0 corrupt: 100%|██████████| 2501/2501 [00:40<00:00, 61.62it/s] 
train: New cache created: /content/drive/MyDrive/HYU/2025/AUE8088/project/AUE8088/datasets/nuscenes/labels/train2007.cache


In [ ]:
for imgs, targets, paths, _, _ in train_loader:
    imgs = imgs.to(device, non_blocking=True).float() / 255  # uint8 to float32, 0-255 to 0.0-1.0
    break

In [ ]:
import torch
from models.common import DetectMultiBackend
from utils.torch_utils import select_device

weights = 'yolov5n.pt'
# data = 'data/nuscenes.yaml'
data = 'data/coco128.yaml'
half = False  # use FP16 half-precision inference
dnn = False  # use OpenCV DNN for ONNX inference
device = select_device('cpu')

model = DetectMultiBackend(weights, device=device, dnn=dnn, data=data, fp16=half)

# inference
model.eval()
with torch.no_grad():
    pred = model(imgs)  # forward

YOLOv5 🚀 2025-5-9 Python-3.11.13 torch-2.6.0+cu124 CPU

100%|██████████| 3.87M/3.87M [00:00<00:00, 96.3MB/s]

Fusing layers... 
YOLOv5n summary: 213 layers, 1867405 parameters, 0 gradients, 4.5 GFLOPs


In [ ]:
from utils.general import non_max_suppression

conf_thres = 0.25  # confidence threshold
iou_thres = 0.45  # NMS IOU threshold
max_det = 1000  # maximum detections per image
classes = None
agnostic_nms = False  # class-agnostic NMS

pred = non_max_suppression(pred, conf_thres, iou_thres, classes, agnostic_nms, max_det=max_det)

# [TODO] draw predictions (see detect.py:L178)
